In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# $\theta_0$ for different components
For foils, $\theta_0$ is fixed by the need for the foil to rotate the given $\lambda_0$ by $\pi$. 

In wollaston prisms and isosceles triangles $\theta_0$ is just a function of the geometry.

For a triangle with base 0.6, height 0.15, the angle is $\arctan(0.15/(0.6/2)) = 0.4636476090008061$. The WSP has the same angle, having just one inclined plane and a base twice as great as the height. 

In [ ]:
from definitions import c

def tune_foil(lambda_0, B_s, d, c, n = 0):
    return np.arcsin(c * d * B_s * lambda_0 / ((2 * n + 1) * np.pi))

B_s = 1
d = 3e-6
# theta_0 = 0.0960
for lambda_0 in [2.165e-10, 3e-10, 4.321e-10, 8e-10, 10e-10]:
    theta_0 = tune_foil(lambda_0, B_s, d, c, n = 0)
    theta_0_deg = np.rad2deg(theta_0)
    phi_foil = c * d * B_s * lambda_0 / np.sin(theta_0)
    print(theta_0, theta_0_deg, phi_foil)

In [ ]:
np.arctan(0.15/0.3)

# $\delta_y$ range for foils
Foils have a set $\theta_0$ as well as a minimal field strength of 0.3mT, giving a natural floor to $\delta_y$ given the other parameters $L_s, \lambda_0$.

In [ ]:
for lambda_0 in [2.165e-10, 4.321e-10, 8e-10]:
    theta_0 = tune_foil(lambda_0, B_s, d, c)
    theta_0_deg = np.rad2deg(theta_0)
    phi_foil = c * d * B_s * lambda_0 / np.sin(theta_0)
    print(theta_0, theta_0_deg, phi_foil)
    L_s = 1.8
    delta_y_min = c * lambda_0 ** 2 * L_s * (0.3e-3) / (np.pi * np.tan(theta_0)) 
    print(delta_y_min)